In [14]:
# Set up interactive plotting using matplotlib, and load numpy
%pylab ipympl
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


# Import PODPAC dependencies

In [15]:
import podpac
from podpac.datalib import smap

# Provide Earth Data Login Credentials
If you do not have an earth data login, follow the [instructions here](https://creare-com.github.io/podpac-docs/user/earthdata.html)

In [3]:
import getpass
username = input("Username:");   password = getpass.getpass('Password:')

# Create the PODPAC SMAP Node

In [16]:
# Create the SMAP node
product = 'SPL4SMAU.003'   # Level 4 soil moisture analysis update
sm = smap.SMAP(product=product, interpolation='nearest', username=username, password=password)

# Retrieve and plot SMAP data for:
## * A particular date
## * Over lat-lon range with user-specified grid

In [17]:
c_world = podpac.Coordinate(#dim=(start, stop, step)
                             lat=(   90,  -90,-2.0),
                             lon=( -180,  180, 2.0), 
                             time='2018-05-19T12:00:00')
o = sm.execute(c_world)
figure()
o.plot()
axis('scaled')

FigureCanvasNbAgg()

(-181.0, 181.0, -91.0, 91.0)

# Retrieve and plot SMAP data for:
## * A date range
## * With user-specified lat-lon points

In [18]:
lat_lon_pts = np.array([[45.0, -100],
                        [45.0, 4],
                        [0.0, 25],
                        [45.0, 100.]]).T
plot(lat_lon_pts[1], lat_lon_pts[0], 'ro')

In [20]:
c_pts = podpac.Coordinate(lat_lon=lat_lon_pts,
                         time=('2018-05-15T00', '2018-05-19T00', '3,h')
                         )
sm.threaded = True; sm.n_threads = 8
o = sm.execute(c_pts)
sm.threaded = False

In [21]:
figure()
plot(o.time, o.data.T)
legend([str(llp) for llp in lat_lon_pts.T])
locs, labels = xticks()
xticks(locs[::2])

FigureCanvasNbAgg()

([<matplotlib.axis.XTick at 0x7fca740656d8>,
 <a list of 5 Text xticklabel objects>)

# Retrieve and plot SMAP data for:
## * A particular date 
## * Over lat-lon range with underlying, native SMAP coordinates
### Define a lat-lon bounding box for retrieving SMAP data

In [9]:
bbox = podpac.Coordinate(#dim=(start, stop, number)
                          lat=(   49,   25, 2),
                          lon=( -126,  -66, 2))

### Get native coordinates of the SMAP data

In [10]:
# !!!!! This next line will take a while first time it is run
sm_nc = sm.native_coordinates
sm_nc

Coordinate
	time: MonotonicCoord: Bounds[2015-03-31T03:00:00, 2018-06-05T00:00:00], N[9296], ctype["segment"]
	lat: MonotonicCoord: Bounds[-84.65644073486328, 84.65644073486328], N[1624], ctype["segment"]
	lon: MonotonicCoord: Bounds[-179.95474243164062, 179.95474243164062], N[3856], ctype["segment"]

In [11]:
# Get the SMAP native coordinates in the desired bounding box
c_intersect = sm_nc.intersect(bbox)
c_intersect.drop_dims('time')
c_intersect

Coordinate
	lat: MonotonicCoord: Bounds[24.945680618286133, 49.05809783935547], N[273], ctype["segment"]
	lon: MonotonicCoord: Bounds[-126.08280944824219, -65.96044921875], N[645], ctype["segment"]

### Select a specific date and time

In [12]:
c = podpac.Coordinate(time='2017-04-30T21:00:00') + c_intersect
c

Coordinate
	time: MonotonicCoord: Bounds[2017-04-30T21:00:00, 2017-04-30T21:00:00], N[1], ctype["segment"]
	lat: MonotonicCoord: Bounds[24.945680618286133, 49.05809783935547], N[273], ctype["segment"]
	lon: MonotonicCoord: Bounds[-126.08280944824219, -65.96044921875], N[645], ctype["segment"]

### Retrieve and plot the data

In [13]:
o = sm.execute(c)
figure()
o.plot()
axis('scaled')

FigureCanvasNbAgg()

(-126.12966918945312,
 -65.91362762451172,
 24.906822204589844,
 49.111507415771484)